## This notebook  does the following:
- processes video frames in the given directory
- uses YOLO version 2 for pedestrian detection in each frame
- uses a custom greedy search algorithm to mathc detections in succesive frames based on procimity and create tracks.
- Saves the traks into a pickle object.

In [1]:
import numpy as np
import time
import Person
import random
from matplotlib.widgets import RectangleSelector
import matplotlib.patches as patches
import cv2
import lightnet
import os
import matplotlib.pyplot as plt
%matplotlib inline 

In [2]:
class Detection:
        def __init__(self, tl, br, cfd):
                self.tl = tl
                self.br = br
                self.confidence = cfd
                self.found= False
        def getTl(self):
                return self.tl
        def getBr(self):
                return self.br
        def getCfd(self):
                return self.confidence
        def setFound(self):
                self.found= True
        def isFound(self):
                return self.found

In [3]:
class Person:
    tracks = []
    def __init__(self, i, tl, br, f):
        self.id = i
        self.tl = tl
        self.br = br
        self.frameId = f
        
        self.tracks = []
        self.frames = []
        self.framesDet = []
        
        self.tracks.append((tl,br))
        self.frames.append(f)
        self.framesDet.append(True)

        self.frameLost = 0
        self.area = abs((self.br[0]-self.tl[0]) * (self.br[1]-self.tl[1]))
        
        self.active = True

    def getTracks(self):
        return self.tracks
    def getId(self):
        return self.id
    def getFrameLost(self):
        return self.frameLost
    def getTl(self):
        return self.tl
    def getBr(self):
        return self.br
    def isActive(self):
        return self.active
    def getMid(self):
        return ((self.tl[0]+ self.br[0])/2 , (self.tl[1]+ self.br[1])/2)
    def getArea(self):
        return self.area       
    def deactivate(self):
        self.active = False

    def getPredictedOffset(self):#how are you predicting?
        l = len(self.tracks)
        if l <= 1:
            return (0,0)
        if l >= 3:
            b=l-3
        else:
            b= l-2
        p2 = cal_mid(self.tracks[l-1][0],self.tracks[l-1][1])
        p1 = cal_mid(self.tracks[b][0],self.tracks[b][1])
        
        deltaX = (p2[0]-p1[0])/(l-b)
        deltaY = (p2[1]-p1[1])/(l-b)
        return (np.int(deltaX), np.int(deltaY))#changed
    
    def walk(self):
        deltaX,deltaY = self.getPredictedOffset()
        self.tl = (self.tl[0] + deltaX, self.tl[1] + deltaY)
        self.br = (self.br[0] + deltaX, self.br[1] + deltaY)
        self.frameId += 1
        self.tracks.append((self.tl,self.br))
        self.frames.append(self.frameId)
        self.framesDet.append(False)
        self.frameLost = self.frameLost+1
        
        if self.frameLost > 10:
            self.deactivate()
        
    def matched(self, tl, br):       
        self.tl = tl
        self.br = br
        self.tracks[len(self.tracks)-1] = (tl,br)
        self.framesDet[len(self.framesDet)-1] = True
        self.frameLost = 0


In [4]:
#video feed
#capture = cv2.VideoCapture('s5.avi')

In [5]:
def cal_distance(p1,p2):
    return np.sqrt( (p2[0] - p1[0])**2 + (p2[1] - p1[1])**2 )

def shift_point(point,shift):
    return ( point[0] + shift[0], point[1] + shift[1] )

def cal_mid(p1,p2):
    return ((p1[0]+ p2[0])/2 , (p1[1]+ p2[1])/2)

In [6]:
def find_all_persons(detections, frame):
        global pid
        for p in [p for p in persons if p.isActive()]:
            maxScore = 0
            score = 0
            for d in [d for d in detections if d.isFound() == False]:                                
                tl = d.getTl()
                br = d.getBr()
                cfd = d.getCfd()
                
                p.walk()
                
                m_offset = cal_distance(p.getMid(), cal_mid(tl,br))
                tl_offset = cal_distance(p.getTl(), tl)
                br_offset = cal_distance(p.getBr(), br)
                
                th = np.sqrt(p.getArea())* offset_threshhold
                if (m_offset < th or tl_offset < th or br_offset < th):
                    score = 1/(1+m_offset+tl_offset+br_offset)
                    if score > maxScore:
                        maxScore = score
                        det = d
                        #print(str(maxScore))
           
                        
            if(maxScore > 0):
                p.matched(det.getTl(), det.getBr())
                det.setFound()

        for d in [d for d in detections if d.isFound() == False and d.getCfd() > 0.6]:
            p = Person(pid,d.getTl(), d.getBr(), frame)
            persons.append(p)
            pid += 1 

In [7]:
def printPersons():
        print('pid: '+str(pid))
        for p in persons:
            print('top-left: '+str(p.getTl())+'top-right:'+str(p.getBr()))

In [8]:
def print_paths(image):
        for p in persons:
                if len(p.getTracks()) > 16 :
                        temp = p.getTracks()[0]
                        a = random.randint(0,254)
                        b = random.randint(0,254)
                        c = random.randint(0,254)
                        cv2.putText(image, str(p.getId()), (int(temp[0]),int(temp[1])), cv2.FONT_HERSHEY_COMPLEX, 1, (a,b,c), 8)
                        for point in p.getTracks():
                                cv2.line(image,(int(temp[0]),int(temp[1])),(int(point[0]),int(point[1])),(a,b,c),3) # draw line between former and present pixel
                                temp = point
                                #cv2.imshow('path', image)
                                #cv2.waitKey(100)
        cv2.imwrite('path.png',image)

In [9]:
def draw_people(image):
        frame = image
        for p in [p for p in persons if p.isActive()]:
                frame = cv2.rectangle(image,p.getTl(),p.getBr(),(0,255,0),7)
                cv2.putText(frame, str(p.getId()), (int(p.getMid()[0]),int(p.getMid()[1])), cv2.FONT_HERSHEY_COMPLEX, 1, (255,0,0), 8)
        return frame

In [10]:
model = lightnet.load('yolo')
#image = lightnet.Image.from_bytes(open('people.jpg', 'rb').read())

In [34]:
demoDir = '/home/urwa/Documents/Projects/tracking/data/StreetDataFrames/7/'
demoImages = os.listdir(demoDir)
demoImages = [img for i,img in sorted(zip([int(img.split('-')[1].split('.')[0]) for img in demoImages],demoImages))]

In [35]:
#outputDir = 'data/StreetDataOutput/5/'

In [36]:
#initialize persons
persons = []
pid = 1


offset_threshhold = 0.5
probThresh = 0.1

height, width = (100,100)

In [37]:
frameID = 0
for img in demoImages:
        image = lightnet.Image.from_bytes(open(demoDir+img, 'rb').read())
        frame = cv2.imread(demoDir+img)
        height, width, channels = frame.shape

        frameID += 1
        printPersons()

        detections = []
        
        height, width, channels = frame.shape
        result = model(image,thresh=probThresh)
        results = [b for b in result if b[1]=='person']
        
        for res in results:
            x,y,w,h = res[3]        
            tl= (int(x-w/2),int(y-h/2))
            br= (int(x+w/2),int(y+h/2))
            cfd = res[2]
            detections.append(Detection(tl, br, cfd))

                
        find_all_persons(detections, frameID)
        #frame = draw_people(frame)
        #cv2.imwrite(outputDir+img,frame)
                

pid: 1
pid: 2
top-left: (213, 133)top-right:(242, 190)
pid: 2
top-left: (213, 133)top-right:(242, 190)
pid: 2
top-left: (213, 133)top-right:(241, 190)
pid: 2
top-left: (213, 132)top-right:(242, 190)
pid: 2
top-left: (213, 133)top-right:(241, 189)
pid: 2
top-left: (212, 131)top-right:(242, 190)
pid: 2
top-left: (211, 132)top-right:(242, 190)
pid: 2
top-left: (212, 131)top-right:(241, 190)
pid: 2
top-left: (211, 130)top-right:(241, 192)
pid: 2
top-left: (210, 131)top-right:(241, 191)
pid: 2
top-left: (210, 130)top-right:(240, 190)
pid: 2
top-left: (209, 129)top-right:(240, 191)
pid: 2
top-left: (209, 129)top-right:(241, 191)
pid: 2
top-left: (210, 129)top-right:(241, 191)
pid: 2
top-left: (210, 129)top-right:(241, 191)
pid: 2
top-left: (209, 130)top-right:(241, 190)
pid: 2
top-left: (207, 130)top-right:(242, 190)
pid: 2
top-left: (207, 131)top-right:(241, 189)
pid: 2
top-left: (208, 129)top-right:(241, 191)
pid: 2
top-left: (207, 128)top-right:(241, 191)
pid: 2
top-left: (208, 129)top-ri

pid: 4
top-left: (204, 113)top-right:(239, 175)
top-left: (327, 81)top-right:(358, 142)
top-left: (533, 112)top-right:(575, 173)
pid: 4
top-left: (204, 113)top-right:(240, 174)
top-left: (326, 81)top-right:(358, 142)
top-left: (533, 110)top-right:(571, 173)
pid: 4
top-left: (204, 113)top-right:(240, 175)
top-left: (326, 81)top-right:(358, 142)
top-left: (533, 109)top-right:(566, 175)
pid: 4
top-left: (204, 112)top-right:(240, 173)
top-left: (326, 80)top-right:(357, 141)
top-left: (531, 109)top-right:(564, 173)
pid: 4
top-left: (206, 111)top-right:(240, 173)
top-left: (326, 81)top-right:(357, 140)
top-left: (531, 106)top-right:(563, 170)
pid: 4
top-left: (204, 111)top-right:(240, 173)
top-left: (326, 80)top-right:(357, 139)
top-left: (530, 106)top-right:(563, 170)
pid: 4
top-left: (203, 111)top-right:(239, 171)
top-left: (324, 78)top-right:(356, 138)
top-left: (529, 105)top-right:(561, 170)
pid: 4
top-left: (203, 110)top-right:(238, 170)
top-left: (325, 78)top-right:(355, 137)
top-left:

pid: 4
top-left: (221, 108)top-right:(254, 172)
top-left: (318, 99)top-right:(348, 154)
top-left: (428, 98)top-right:(465, 155)
pid: 4
top-left: (221, 109)top-right:(255, 174)
top-left: (317, 99)top-right:(348, 154)
top-left: (429, 98)top-right:(461, 155)
pid: 4
top-left: (221, 111)top-right:(255, 176)
top-left: (318, 100)top-right:(348, 154)
top-left: (432, 91)top-right:(459, 152)
pid: 4
top-left: (222, 110)top-right:(256, 177)
top-left: (318, 100)top-right:(348, 154)
top-left: (432, 91)top-right:(459, 153)
pid: 4
top-left: (221, 110)top-right:(256, 177)
top-left: (319, 102)top-right:(348, 153)
top-left: (430, 91)top-right:(457, 153)
pid: 4
top-left: (221, 111)top-right:(256, 176)
top-left: (318, 102)top-right:(347, 153)
top-left: (429, 90)top-right:(457, 153)
pid: 4
top-left: (221, 111)top-right:(256, 176)
top-left: (319, 103)top-right:(348, 153)
top-left: (419, 90)top-right:(448, 155)
pid: 4
top-left: (221, 110)top-right:(256, 176)
top-left: (318, 101)top-right:(347, 155)
top-left: 

pid: 4
top-left: (218, 105)top-right:(247, 168)
top-left: (281, 111)top-right:(316, 170)
top-left: (318, 90)top-right:(334, 135)
pid: 4
top-left: (218, 105)top-right:(247, 168)
top-left: (281, 110)top-right:(316, 171)
top-left: (316, 90)top-right:(333, 136)
pid: 4
top-left: (218, 105)top-right:(246, 169)
top-left: (281, 111)top-right:(317, 171)
top-left: (312, 89)top-right:(327, 134)
pid: 4
top-left: (218, 105)top-right:(246, 166)
top-left: (279, 109)top-right:(318, 171)
top-left: (306, 84)top-right:(322, 125)
pid: 4
top-left: (218, 105)top-right:(246, 165)
top-left: (280, 111)top-right:(317, 169)
top-left: (301, 84)top-right:(319, 126)
pid: 4
top-left: (217, 104)top-right:(247, 163)
top-left: (279, 112)top-right:(317, 169)
top-left: (296, 77)top-right:(322, 140)
pid: 4
top-left: (218, 103)top-right:(246, 163)
top-left: (280, 110)top-right:(317, 166)
top-left: (294, 79)top-right:(319, 146)
pid: 4
top-left: (218, 103)top-right:(246, 164)
top-left: (279, 109)top-right:(316, 166)
top-left

pid: 5
top-left: (218, 97)top-right:(243, 164)
top-left: (209, 80)top-right:(224, 116)
top-left: (213, 110)top-right:(240, 177)
top-left: (251, 124)top-right:(281, 187)
pid: 5
top-left: (217, 98)top-right:(243, 165)
top-left: (210, 82)top-right:(225, 114)
top-left: (213, 123)top-right:(240, 190)
top-left: (249, 125)top-right:(281, 186)
pid: 5
top-left: (216, 99)top-right:(244, 166)
top-left: (210, 85)top-right:(224, 113)
top-left: (221, 117)top-right:(237, 149)
top-left: (252, 131)top-right:(282, 190)
pid: 5
top-left: (217, 99)top-right:(244, 164)
top-left: (210, 85)top-right:(224, 113)
top-left: (221, 114)top-right:(237, 149)
top-left: (253, 133)top-right:(282, 188)
pid: 5
top-left: (216, 102)top-right:(245, 164)
top-left: (210, 85)top-right:(224, 113)
top-left: (220, 114)top-right:(237, 149)
top-left: (252, 134)top-right:(282, 188)
pid: 5
top-left: (216, 102)top-right:(245, 165)
top-left: (210, 85)top-right:(224, 113)
top-left: (220, 114)top-right:(237, 150)
top-left: (251, 134)top-r

...

pid: 14
top-left: (278, 86)top-right:(313, 146)
top-left: (210, 85)top-right:(224, 113)
top-left: (197, 170)top-right:(224, 232)
top-left: (219, 121)top-right:(238, 156)
top-left: (55, -14)top-right:(106, 94)
top-left: (60, 159)top-right:(91, 216)
top-left: (98, 277)top-right:(144, 349)
top-left: (545, 72)top-right:(581, 118)
top-left: (300, 54)top-right:(315, 82)
top-left: (0, 306)top-right:(16, 345)
top-left: (50, 288)top-right:(93, 351)
top-left: (396, 52)top-right:(430, 106)
top-left: (552, 84)top-right:(574, 110)
pid: 14
top-left: (279, 86)top-right:(313, 146)
top-left: (210, 85)top-right:(224, 113)
top-left: (197, 170)top-right:(224, 232)
top-left: (219, 121)top-right:(238, 156)
top-left: (55, -14)top-right:(106, 94)
top-left: (60, 159)top-right:(91, 216)
top-left: (98, 277)top-right:(144, 349)
top-left: (546, 72)top-right:(583, 117)
top-left: (300, 54)top-right:(315, 82)
top-left: (0, 306)top-right:(16, 345)
top-left: (50, 288)top-right:(93, 351)
top-left: (396, 52)top-right:(43

pid: 14
top-left: (271, 89)top-right:(307, 149)
top-left: (210, 85)top-right:(224, 113)
top-left: (197, 170)top-right:(224, 232)
top-left: (219, 121)top-right:(238, 156)
top-left: (55, -14)top-right:(106, 94)
top-left: (60, 159)top-right:(91, 216)
top-left: (98, 277)top-right:(144, 349)
top-left: (564, 73)top-right:(598, 126)
top-left: (300, 54)top-right:(315, 82)
top-left: (0, 306)top-right:(16, 345)
top-left: (50, 288)top-right:(93, 351)
top-left: (396, 52)top-right:(430, 106)
top-left: (575, 87)top-right:(592, 114)
pid: 14
top-left: (271, 88)top-right:(306, 149)
top-left: (210, 85)top-right:(224, 113)
top-left: (197, 170)top-right:(224, 232)
top-left: (219, 121)top-right:(238, 156)
top-left: (55, -14)top-right:(106, 94)
top-left: (60, 159)top-right:(91, 216)
top-left: (98, 277)top-right:(144, 349)
top-left: (565, 73)top-right:(598, 125)
top-left: (300, 54)top-right:(315, 82)
top-left: (0, 306)top-right:(16, 345)
top-left: (50, 288)top-right:(93, 351)
top-left: (396, 52)top-right:(43

pid: 14
top-left: (274, 91)top-right:(301, 151)
top-left: (210, 85)top-right:(224, 113)
top-left: (197, 170)top-right:(224, 232)
top-left: (219, 121)top-right:(238, 156)
top-left: (55, -14)top-right:(106, 94)
top-left: (60, 159)top-right:(91, 216)
top-left: (98, 277)top-right:(144, 349)
top-left: (591, 80)top-right:(625, 130)
top-left: (300, 54)top-right:(315, 82)
top-left: (0, 306)top-right:(16, 345)
top-left: (50, 288)top-right:(93, 351)
top-left: (396, 52)top-right:(430, 106)
top-left: (600, 86)top-right:(620, 123)
pid: 14
top-left: (274, 93)top-right:(300, 150)
top-left: (210, 85)top-right:(224, 113)
top-left: (197, 170)top-right:(224, 232)
top-left: (219, 121)top-right:(238, 156)
top-left: (55, -14)top-right:(106, 94)
top-left: (60, 159)top-right:(91, 216)
top-left: (98, 277)top-right:(144, 349)
top-left: (593, 81)top-right:(624, 134)
top-left: (300, 54)top-right:(315, 82)
top-left: (0, 306)top-right:(16, 345)
top-left: (50, 288)top-right:(93, 351)
top-left: (396, 52)top-right:(43

pid: 14
top-left: (273, 101)top-right:(295, 154)
top-left: (210, 85)top-right:(224, 113)
top-left: (197, 170)top-right:(224, 232)
top-left: (219, 121)top-right:(238, 156)
top-left: (55, -14)top-right:(106, 94)
top-left: (60, 159)top-right:(91, 216)
top-left: (98, 277)top-right:(144, 349)
top-left: (604, 85)top-right:(637, 138)
top-left: (300, 54)top-right:(315, 82)
top-left: (0, 306)top-right:(16, 345)
top-left: (50, 288)top-right:(93, 351)
top-left: (396, 52)top-right:(430, 106)
top-left: (612, 95)top-right:(627, 120)
pid: 14
top-left: (273, 102)top-right:(295, 154)
top-left: (210, 85)top-right:(224, 113)
top-left: (197, 170)top-right:(224, 232)
top-left: (219, 121)top-right:(238, 156)
top-left: (55, -14)top-right:(106, 94)
top-left: (60, 159)top-right:(91, 216)
top-left: (98, 277)top-right:(144, 349)
top-left: (607, 83)top-right:(636, 138)
top-left: (300, 54)top-right:(315, 82)
top-left: (0, 306)top-right:(16, 345)
top-left: (50, 288)top-right:(93, 351)
top-left: (396, 52)top-right:(

pid: 14
top-left: (266, 100)top-right:(293, 157)
top-left: (210, 85)top-right:(224, 113)
top-left: (197, 170)top-right:(224, 232)
top-left: (219, 121)top-right:(238, 156)
top-left: (55, -14)top-right:(106, 94)
top-left: (60, 159)top-right:(91, 216)
top-left: (98, 277)top-right:(144, 349)
top-left: (631, 93)top-right:(640, 138)
top-left: (300, 54)top-right:(315, 82)
top-left: (0, 306)top-right:(16, 345)
top-left: (50, 288)top-right:(93, 351)
top-left: (396, 52)top-right:(430, 106)
top-left: (620, 72)top-right:(643, 142)
pid: 14
top-left: (266, 100)top-right:(293, 158)
top-left: (210, 85)top-right:(224, 113)
top-left: (197, 170)top-right:(224, 232)
top-left: (219, 121)top-right:(238, 156)
top-left: (55, -14)top-right:(106, 94)
top-left: (60, 159)top-right:(91, 216)
top-left: (98, 277)top-right:(144, 349)
top-left: (632, 90)top-right:(640, 141)
top-left: (300, 54)top-right:(315, 82)
top-left: (0, 306)top-right:(16, 345)
top-left: (50, 288)top-right:(93, 351)
top-left: (396, 52)top-right:(

In [38]:
import pickle
with open('street_7.pkl', 'wb') as output:
    pickle.dump(persons, output, pickle.HIGHEST_PROTOCOL)

In [87]:
# with open('street_2.pkl', 'rb') as input:
#     persons = pickle.load(input)

In [88]:
height,width

(352, 640)